# Exercice: Modélisation orientée requête (Cassandra)

<table><tr>
    <td><img src="image/esi-sba.png" width="100" height="100"></td>
    <td><img src="image/cassandra.png" width="100" height="100"></td>
    </tr></table>



### Dans ce notebook, nous allons travailler sur la modélisation orientée requête sous la BDD No-SQL Cassandra, en particulier nous traitons les points suivants :

* Dénormaliser une BDD relationnelle en partant du principe 1 table per query 
* Définir la partie Primary Key pour chaque requête
* Optimiser vos modèles en choisissant la bonne Partition Key & Clustering Column
* Valider vos modèles 
* Tester quelques requêtes particulières


#### installer cassandra-driver afin de se connecter au cluster cassandra
! pip install cassandra-driver
#### More documentation can be found here:  https://datastax.github.io/python-driver/

#### Import Apache Cassandra python package

In [1]:
import cassandra

### créer une connection au cluster cassandra

In [2]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) # si cassandra est installé localement avec le port par défaut 9042
    session = cluster.connect()
except Exception as e:
    print(e)

### créer maintenant un keyspace nommé "iot_esi"

In [3]:
try:
    
  ### add code here 
  session.execute("""
    CREATE KEYSPACE IF NOT EXISTS iot_esi3
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""") 

except Exception as e:
    print(e)

### connecter à votre Keyspace  iot_esi 


In [4]:
try:
    
    session.set_keyspace('iot_esi3')
    
except Exception as e:
    print(e)

### Supposons qu'on a un système IoT qui stocke les données capturées par les différents Sensors en utilisant un modèle RELATIONNEL, tel que est décrit dans le schéma ER ci-dessous:

<table><tr>
    <td><img src="image/ex1_IoT_ER_model.png" width="500" height="500"></td>
      </tr></table>

### imaginons  que nous souhaitons analyser ces données à  travers 3 requêtes analytiques:
* `Query1= retourner les informations(name, region, nbSensors) de tous les Networks`
* `Query2= retourner les informations (name,type,localisation) de tous les Sensors d'un Network donné.`
* `Query3= retourner le AVG_measurements par sensor,  par jour et par heure.  Ordonner le résultat par  jour(desc) et par heure (asc)`

#### Créer les trois tables qui correspondent aux requêtes précédentes, en indiquant pour chacune:
* Primary Key
* Partition Key
* Clustering Column

In [5]:
try:
    
    ### Create the table that corresponds to the Query1
    query1=" create table if not exists networks (nameN text, region text, nbSensors int, "
    query1=query1+" primary key(region, nameN))"
    session.execute(query1)

    ### Create the table that corresponds to the Query2
    query2="create table if not exists Sensor_By_Network (nameN text, nameS text, type text,"
    query2 = query2 +"  latitude float, longitude float, primary key(nameN,nameS))"
    session.execute(query2)
    
    
    ### Create the table that corresponds to the Query3
    query3="create table if not exists AVG_measurements (nameS text, jour date, hour time, "
    query3=query3+" AVG_measurements float, primary key(nameS,jour,hour))"
    query3=query3+" with clustering order by (jour desc, hour asc) "
    session.execute(query3)
    
    
except Exception as e:
        print(e)

### Une fois vous créez les différentes tables, essayez de les peupler en se basant sur le contenu de la BDD relationnelle.

<table><tr>
    <td><img src="image/network_data.png" width="200" height="300"></td></tr>
   <tr> <td><img src="image/sensor_data.png" width="400" height="500"></td> </tr>
        <tr>
       <td><img src="image/recorded_data.png" width="350" height="250"></td>
      </tr>
</table>

### Insertion des données dans Table1

In [6]:
query = "INSERT INTO networks (nameN, region, nbSensors) "
query = query + " VALUES (%s,%s, %s)"

try:
    session.execute(query, ("esi-net","ouest",1))
    session.execute(query, ("udl-net","ouest",1))
    session.execute(query, ("alger-net","center",0))
except Exception as e:
        print(e)  

### Insertion des données dans Table2

In [7]:
query = "INSERT INTO Sensor_By_Network (nameN , nameS , type , latitude, longitude)"
query = query + " VALUES (%s, %s, %s, %s, %s)"

try:
    session.execute(query, ("esi-net","temp-454","temp",30.526503,-95.592817))
    session.execute(query, ("udl-net","temp-556232","temp",31.526503,-96.592817))
except Exception as e :
        print(e)      

### Insertion des données dans Table3

In [8]:
query = "INSERT INTO  AVG_measurements (nameS,  jour, hour , AVG_measurements)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, ("temp-454","2021-10-31","11:00:00", 10))
    session.execute(query, ("temp-454","2021-10-31","12:00:00", 11.8))
    session.execute(query, ("temp-454","2021-10-31","13:00:00", 12.25))
    session.execute(query, ("temp-556232","2021-10-31","11:00:00", 24))
    session.execute(query, ("temp-556232","2021-10-31","14:00:00", 26))
except Exception as e :
        print(e)    

### Valider votre modèle en exécutant les requêtes suivantes

* Retourner le nombre de sensors dans la région ouest"

* Retourner les sensors qui sont placés dans le network "esi-net"

* Retourner la température maximale enregistrée par le sensor "temp-454" le 31-10-2021

In [ ]:
# query1
query = "......"

try:
    rows = session.execute(query)
    for row in rows:
          print (....)
except Exception :e
        print(e)            

In [ ]:
# query2
query = "......"

try:
    rows = session.execute(query)
    for row in rows:
          print (....)
except Exception :e
        print(e)            

In [ ]:
# query3
query = "......"

try:
    rows = session.execute(query)
    for row in rows:
          print (....)
except Exception :e
        print(e)            

### Essayons d'autres requêtes 

* `Retourner la température maximale enregistrée par le sensor "temp-454" à 13:00:00 pendant le  mois d'octobre` 

In [ ]:
# query4
query = "...."


try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)      

### Proposer une solution pour optimiser cette requête

In [ ]:
### make your solution 


###  drop  tables. 

In [ ]:
query = "....."
rows = session.execute(query)

query = "....."
rows = session.execute(query)

query = "....."
rows = session.execute(query)


### close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()